In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gensim.models import Word2Vec
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
dataset = pd.read_csv(r'news_articles.tsv', delimiter='\t', quoting=3, encoding='ISO-8859-1')
article = dataset.iloc[:,1]
topic = dataset.iloc[:,2]

In [4]:
corpus=[]
for i in range(len(article)):
    review=re.sub('[^a-zA-Z]',' ', article[i])
    review=review.lower()
    review=review.split()
    review=[w for w in review if not w in set(stopwords.words('english'))]
    corpus.append(review)

In [5]:
model=Word2Vec(corpus, min_count=1)

In [6]:
a=list(model.wv.vocab)
sum_paris=model.wv['paris']
sum_paris

array([ 8.64057317e-02,  6.46250471e-02, -6.03719912e-02,  2.28480455e-02,
       -5.32859825e-02, -1.71772223e-02,  8.10649619e-03, -3.82137150e-02,
        3.75701077e-02,  8.38636234e-03,  7.64902914e-03, -4.72657755e-02,
        4.28405367e-02,  7.56360299e-04, -4.26443778e-02,  4.14133817e-02,
        1.34647321e-02,  6.51616380e-02,  7.10929325e-03, -2.68281624e-02,
        2.42114998e-02,  4.94904518e-02, -1.14265740e-01, -5.89296632e-02,
        1.75156631e-02,  6.69979975e-02, -1.81135815e-03, -2.09504738e-02,
       -2.12938003e-02, -1.05127590e-02,  1.11264214e-02, -2.52294242e-02,
        5.38553223e-02,  2.50968020e-02,  5.75859211e-02, -5.44535369e-02,
        3.90993766e-02,  7.72871077e-02,  2.87212860e-02,  1.62216760e-02,
        4.11939248e-02, -4.75209653e-02,  8.93843323e-02, -3.12169660e-02,
        7.01026917e-02, -2.06247251e-03, -5.04766628e-02,  7.17597902e-02,
        4.69034712e-05,  9.06722322e-02, -3.84122021e-02,  8.91817082e-03,
        8.02364871e-02,  

In [7]:
master_word_list=[]
master_article=''
for j in range(len(article)):
    master_article+=re.sub('[^a-zA-Z]',' ',article[j])

In [8]:
stop_words=set(stopwords.words('english'))
master_word_list=master_article.lower().split()
word_tokens=word_tokenize(master_article.lower())
master_nouns=[w for w in word_tokens if not w in stop_words]
#master_tagged_words=pos_tag(master_filtered_sentence)
#master_nouns=[word for word, pos in master_tagged_words\
              #if(pos == 'NN' or pos == 'NNP' or pos == 'NNS')]
master_word_freq=[]

for k in range(len(master_nouns)):
        master_word_freq.append(master_word_list.count(master_nouns[k]))
idf=[]
for i in range(len(master_nouns)):
    score=0
    for j in range(len(article)):
        if(master_nouns[i] in article[j]):
            score+=1
    idf.append(score)    
dict_article={
      'data':master_nouns,
      'freq':master_word_freq,
      'idf':idf
     }  
columns=['data','freq','idf']     
master_set=pd.DataFrame(dict_article, columns=columns)
master_set=master_set.loc[master_set['freq']!=0]
master_set=master_set.loc[master_set['idf']!=0]
master_set=master_set.drop_duplicates().reset_index()
master_set=master_set.drop(['index'],axis=1)
master_set.iloc[:,1]=master_set.iloc[:,1].values.astype(np.float64)
master_set.iloc[:,2]=master_set.iloc[:,2].values.astype(np.float64)
tot=np.sum(master_set.iloc[:,1])
vocab=master_set.iloc[:,0]
master_set

,data,freq,idf
0,paris,12.0,7.0
1,federal,15.0,3.0
2,reserve,8.0,10.0
3,bank,81.0,23.0
4,president,21.0,6.0
5,votes,1.0,1.0
6,policy,21.0,16.0
7,year,129.0,88.0
8,said,358.0,132.0
9,recent,42.0,45.0


In [9]:
final_data=[0]*8389
for l in range (len(article)):
    article_data=re.sub('[^a-zA-Z]',' ',article[l])
    stop_words=set(stopwords.words('english'))
    wordlist=article_data.lower().split()
    word_tokens=word_tokenize(article_data.lower())
    nouns = [w for w in word_tokens if not w in stop_words]
    word_freq = []
    for i in range(len(nouns)):
        word_freq.append(wordlist.count(nouns[i]))
    dict_data=dict(zip(nouns,word_freq))
    dict_data={k:v for k,v in dict_data.items() if v!=0}
    #we need to get the term frequency of each word in the document, and then take tf represents the log probability.
    presence=[]
    for master in range(len(master_set)):
        if(master_set.iloc[master,0] in dict_data):
            #appending '1' for the words which are present.
            alpha=(dict_data.get(master_set.iloc[master,0])/master_set.iloc[master,1])
            value=alpha*np.abs(np.sum(model.wv[master_set.iloc[master,0]]))
            presence.append(value)
        else:
            #appending '0' for the words which are not present.
            presence.append(0)
    #we need to get the tf_idf for each word in the document, and then take the sum of it.
    final_data=np.column_stack((final_data,presence))

In [10]:
final_data=final_data[:,1:257]

In [11]:
master_column=master_set.iloc[:,0].tolist()
master_column

['paris',
 'federal',
 'reserve',
 'bank',
 'president',
 'votes',
 'policy',
 'year',
 'said',
 'recent',
 'encouraging',
 'economic',
 'data',
 'enough',
 'change',
 'pace',
 'central',
 'asset',
 'purchases',
 'would',
 'nice',
 'perspective',
 'started',
 'faster',
 'television',
 'interview',
 'referring',
 'fed',
 'tapering',
 'stimulus',
 'programme',
 'given',
 'fact',
 'embarked',
 'measured',
 'reductions',
 'important',
 'give',
 'certainty',
 'least',
 'clarity',
 'markets',
 'ok',
 'continue',
 'billion',
 'hurdle',
 'rate',
 'pretty',
 'high',
 'either',
 'direction',
 'trying',
 'determine',
 'whether',
 'economy',
 'shown',
 'signs',
 'weakness',
 'severe',
 'winter',
 'weather',
 'fundamental',
 'obstacles',
 'growth',
 'setting',
 'open',
 'market',
 'committee',
 'meets',
 'march',
 'first',
 'gathering',
 'led',
 'chair',
 'since',
 'succeeded',
 'ben',
 'last',
 'month',
 'new',
 'c',
 'sees',
 'reasonably',
 'favorable',
 'outlook',
 'even',
 'unemployment',
 'low

In [12]:
master_data=pd.DataFrame(data=final_data.transpose(),columns=master_column)

In [13]:
master_data

,paris,federal,reserve,bank,president,votes,policy,year,said,recent,...,swearing,steered,distractingly,repetitive,refining,sessions,repopulated,ranked,teens,reacting
0,0.130552,0.125897,0.067158,0.261134,0.209213,0.041461,0.232528,0.056378,0.145185,0.132707,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.062949,0.067158,0.065284,0.069738,0.000000,0.000000,0.112756,0.058074,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,0.065276,0.125897,0.134317,0.065284,0.069738,0.000000,0.155019,0.056378,0.087111,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.261134,0.000000,0.000000,0.077509,0.169133,0.145185,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.195851,0.000000,0.000000,0.077509,0.056378,0.058074,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.456985,0.000000,0.000000,0.000000,0.056378,0.029037,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.062949,0.067158,0.391702,0.069738,0.000000,0.232528,0.281889,0.174223,0.066353,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.130567,0.000000,0.000000,0.077509,0.112756,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.195851,0.000000,0.000000,0.000000,0.056378,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


In [14]:
sum_paris=np.sum(model.wv['paris'])
sum_paris

0.78331226

In [15]:
from sklearn.preprocessing import LabelEncoder
labelEncoder=LabelEncoder()
topic=labelEncoder.fit_transform(topic)
topic

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [16]:
classifier = MultinomialNB(alpha=0.01)
from sklearn import metrics, cross_validation
predicted=cross_validation.cross_val_predict(classifier, master_data, topic, cv=2)
print (metrics.accuracy_score(topic,predicted))
print (metrics.classification_report(topic,predicted))

0.8828125
             precision    recall  f1-score   support

          0       0.90      0.84      0.87        56
          1       0.95      0.90      0.92        67
          2       0.88      0.80      0.84        61
          3       0.82      0.97      0.89        72

avg / total       0.89      0.88      0.88       256



C:\Users\anusengupta\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
